In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset= pd.read_csv('../input/google-stock-prediction/GOOG.csv')

In [ ]:
Google_dataset_train = dataset.iloc[:1247,:]
Google_dataset_test = dataset.iloc[1247:,:]

In [ ]:
training_set = Google_dataset_train.loc[:,['close', 'open']].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
x_train = []
y_train = []

for i in range(60, 1247):
    x_train.append(training_set_scaled[i-60:i,0:2])
    y_train.append(training_set_scaled[i,0:2])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
regressor = Sequential()

In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 2)))
regressor.add(Dropout(0.2))             

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 2))

In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
regressor.fit(x_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
dataset_test = Google_dataset_test.loc[:,['close', 'open']]

dataset_total = dataset.loc[:,['close', 'open'],]
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-2, 2)
inputs = sc.transform(inputs)

x_test = []

for i in range(60, 70):
    x_test.append(inputs[i-60:i,0:2])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 2))
predicted_stock_price = regressor.predict(x_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

real_stock_price = Google_dataset_test.loc[:,['close', 'open']].values


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(predicted_stock_price[:,0], color = 'blue', label = 'Predicted Google stock price(close)')
plt.plot(predicted_stock_price[:,1], color = 'black', label = 'Predicted Google stock price(open)')
plt.plot(real_stock_price[:, 0], color = 'red', label = 'Real Google stock price(close)')
plt.plot(real_stock_price[:, 1], color = 'yellow', label = 'Real Google stock price(open)')
plt.title('Google stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()
